In [9]:
import openpyxl
import re
import requests
import xmltodict
import json
import time

In [10]:
def extract_author_ids_from_excel(file_path, sheet_name):
    author_ids = []
    workbook = openpyxl.load_workbook(file_path)
    sheet = workbook[sheet_name]
    for row in sheet.iter_rows(values_only=True):
        cell_value = row[2]  
        match = re.search(r'authorId=(\d+)', str(cell_value))
        if match:
            author_ids.append(match.group(1))
    return author_ids
author_ids = extract_author_ids_from_excel("D:\Scopus\Profiles links.xlsx", "Sheet1")
print(len(author_ids))

99


In [11]:
def url_Links(url_1,url_2, url_3):
    payload = {}
    headers = {
    'Cookie': '__cf_bm=UBulqlF4MHGnaj4GDKP2G2Um_LtiPItnnhRuW8D7gFw-1709805123-1.0.1.1-zjk_bXUzTJmweWvqwTznPqeRhAbyMbqO4lYLW7OVl9P9zOE4g.QnZa7ew8zE3Xqxw1TuvDs_.CRpruAT6wbapQ'
    }
    response = requests.request("GET", url_1, headers=headers, data=payload)
    response_author = requests.request("GET", url_2,headers= headers,data=payload)
    while response.status_code != 200 or response_author.status_code != 200:
        response = requests.request("GET", url_1, headers=headers, data=payload)
        response_author = requests.request("GET", url_2,headers= headers,data=payload)
        print(response)
        print(response_author)
    else:
        with open('D:\Scopus\Scopus_output\data_H-index.json', 'w') as f:
            json.dump(response.text, f)
        with open('D:\Scopus\Scopus_output\data_author.json', 'w') as f:
            json.dump(response_author.text, f)
        #### ------------------ Convert json to xml & parse ----------------------------- ####
            
        with open("D:\Scopus\Scopus_output\data_H-index.json", "r") as f:
            json_data = json.load(f)
        xml_dict = {"data": json_data}  
        xml_data = xmltodict.unparse(xml_dict, pretty=True)
        with open("D:\Scopus\Scopus_output\data_H-index.xml", "w") as f:
            f.write(xml_data)

        with open("D:\Scopus\Scopus_output\data_author.json", "r") as f1:
            json_data_author = json.load(f1)
        xml_dict_author = {"data": json_data_author}  
        xml_data_author = xmltodict.unparse(xml_dict_author, pretty=True)
        with open("D:\Scopus\Scopus_output\data_author.xml", "w") as f:
            f.write(xml_data_author)

In [12]:
import json
import requests
import xmltodict 
import openpyxl
import pandas as  pd 
df = pd.DataFrame(columns=["Author_inital", "h-index", "citation", "citation_citied_by_documents", "Author_ID", "document_count", "eid", "orcid", "areas_abv", "Publications_Title"])
for i in author_ids:
    url_metrics = f"https://api.elsevier.com/content/author/author_id/{i}?apiKey=89a1ebfbce5e0a87b9c189b908fed168&view=metrics"
    url_author = f"https://api.elsevier.com/content/author/author_id/{i}?apiKey=7649a14c68f2ae7273a0f9a07a1f31b4"
    url_publications = f'https://api.elsevier.com/content/search/scopus?query=au-id%28{i}%29&apiKey=af1361e4e58683804f723b56fed20175'
    url_Links(url_metrics,url_author,url_publications)
    #### ------------------ open xml ----------------------------- ####
    with open("D:\Scopus\Scopus_output\data_H-index.xml", "r") as xml_file:
        data_dict = xmltodict.parse(xml_file.read())
    data_xml = data_dict['data']
    data_dict_parsed = xmltodict.parse(data_xml)
    try:
        H_index_data = data_dict_parsed['author-retrieval-response']['h-index']
        print("H-index", H_index_data)
        citation = data_dict_parsed['author-retrieval-response']['coredata']['citation-count']
        print("citition", citation)
        citation_citied_by_documents = data_dict_parsed['author-retrieval-response']['coredata']['cited-by-count']
        print(citation_citied_by_documents)
        Author_id = data_dict_parsed['author-retrieval-response']['coredata']['dc:identifier']
        print(Author_id)
        document_count = data_dict_parsed['author-retrieval-response']['coredata']['document-count']
        print(document_count)
        co_author_count = data_dict_parsed['author-retrieval-response']['coauthor-count']
        print(co_author_count)
    except KeyError:
        H_index_data = 'NA'
        citation = "NA"
        citation_citied_by_documents = "NA"
        Author_id = "NA"
        document_count = "NA"
        co_author_count = "NA"
        #------------------------ subject area ----------------------------------#
    with open("D:\Scopus\Scopus_output\data_author.xml", "r") as xml_file_author:
        data_dict_author = xmltodict.parse(xml_file_author.read())
    data_xml_author = data_dict_author['data']
    data_dict_parsed_author = xmltodict.parse(data_xml_author)
    try:
        subject_areas = data_dict_parsed_author['author-retrieval-response']['subject-areas']['subject-area']
        if not isinstance(subject_areas, list):
            subject_areas = [subject_areas]
        subject_areas_info = [{'abbrev': area['@abbrev'], 'name': area['#text']} for area in subject_areas]
        #---------
        subject_area_abbrevs = [area['abbrev'] for area in subject_areas_info]
        Area_list =[]
        for area in subject_areas_info:
            Area_list.append(area['name']+ "("+ area['abbrev']+")")
            areas_abv_wth_name = ', '.join(Area_list) 
            areas_abv = ', '.join(subject_area_abbrevs)
  

    except KeyError:
        areas_name = 'NA'
 
    Author_name_init = []
   
    eid = data_dict_parsed_author['author-retrieval-response']['coredata']['eid']
    try:
        orcid = data_dict_parsed_author['author-retrieval-response']['coredata']['orcid']
    except KeyError:
        orcid = 'NA'
    try:
        Author_name = data_dict_parsed_author['author-retrieval-response']['author-profile']['name-variant']
    except KeyError:
        Author_name = 'NA'
    print("orcid", orcid)

    if Author_name == 'NA':
        try:
            Author_name = data_dict_parsed_author['author-retrieval-response']['author-profile']['preferred-name']
            Author_name_init.append(i['surname']+ "," + i['initials'])  
        except KeyError:
            Author_name = 'NA'
    if Author_name != 'NA':
        try: 
            for i in Author_name: 
                        Author_name_init.append(i['surname']+ "," + i['initials'])    
                        print(Author_name_init)
        except TypeError:
            Author_name_init.append(Author_name['surname']+ "," + Author_name['initials'])  

    if (len(Author_name_init) !=0):       
        Name =Author_name_init[0]
    else:
        Name = 'NA'
        #---------------------------------------------------------------------------
    parts = Author_id.split(':')
    if len(parts) == 2:
        number = parts[1].strip()

    #------------------------------------------------------------------------
    response_publication = requests.get(url_publications)
    data = response_publication.json()
    try:
        eid = data['search-results']['entry']
        eids = [item['eid'] for item in eid]
        eids_list = ', '.join(eids)
        titles = [item['dc:title'] for item in eid]
        titles_related_to_eids = ', '.join(titles)
    except KeyError:
        eid = "NA"
        titles = "NA"
        

 
    

    row_data = {
        "Author_inital": Name,
        "h-index": H_index_data,
        "citation": citation,
        "citation_citied_by_documents": citation_citied_by_documents,
        "Author_ID": number,
        "document_count": document_count,
        "eid": eids_list,
        "orcid": orcid,
        "areas_abv": areas_abv,
        "Publications_Title": titles_related_to_eids
    }
    df = df._append(row_data, ignore_index=True)
    df.to_excel('D:\Scopus\Scopus_output\scopus_author_metrics.xlsx')
            



<Response [200]>
<Response [200]>
H-index 3
citition 38
38
AUTHOR_ID:8503356600
17
18
orcid NA
['Awny,M.']
['Awny,M.', 'Awny,M.M.']
['Awny,M.', 'Awny,M.M.', 'Awny,M.M.']
<Response [200]>
<Response [200]>
H-index 53
citition 10146
4613
AUTHOR_ID:7103379659
463
236
orcid 0000-0001-6079-4138
['Radwan,A.']
['Radwan,A.', 'Radwan,A.G.G.']
['Radwan,A.', 'Radwan,A.G.G.', 'Radwand,A.G.']
['Radwan,A.', 'Radwan,A.G.G.', 'Radwand,A.G.', 'Radwan,A.G.']
['Radwan,A.', 'Radwan,A.G.G.', 'Radwand,A.G.', 'Radwan,A.G.', 'Radwan,A.G.']
['Radwan,A.', 'Radwan,A.G.G.', 'Radwand,A.G.', 'Radwan,A.G.', 'Radwan,A.G.', 'Radwan,A.G.']
['Radwan,A.', 'Radwan,A.G.G.', 'Radwand,A.G.', 'Radwan,A.G.', 'Radwan,A.G.', 'Radwan,A.G.', 'Radwan,A.']
['Radwan,A.', 'Radwan,A.G.G.', 'Radwand,A.G.', 'Radwan,A.G.', 'Radwan,A.G.', 'Radwan,A.G.', 'Radwan,A.', 'Ahmed,A.G.']
['Radwan,A.', 'Radwan,A.G.G.', 'Radwand,A.G.', 'Radwan,A.G.', 'Radwan,A.G.', 'Radwan,A.G.', 'Radwan,A.', 'Ahmed,A.G.', 'Radwan,A.G.']
<Response [200]>
<Response [4